# Motif Enrichment
- The motif hits matrix save the max loglikelyhood score of a motif in that DMR
- Before run the motif enrichment, I filter the scores for each motif using cutoff defined by
    - `np.quantile(motif_score[motif_score > 0]) * mask_quantile_to99`
- This filter will remove hits with low score, which is likely to be false positive
- Its likely boosting the significance of real enrichment

In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from scipy.sparse import csr_matrix
from statsmodels.stats.multitest import multipletests
import pathlib

In [2]:
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = 'IT-L23+ACA'

motif_hits_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad'
dmr_hits_path = f'/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/{group}.DMS1.bed'
output_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment'


In [3]:
# Parameters
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = "IT-L23+MOs"
motif_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad"
dmr_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/IT-L23+MOs.DMS1.bed"
output_dir = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment"


In [4]:
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [5]:
dmr_bed_df = pd.read_csv(dmr_hits_path, sep='\t', header=None, index_col=3)
dmr_bed_df.head()

,0,1,2
3,,,
DGm3c_34,chr1,3237224,3237224
DGm3c_43,chr1,3297316,3297413
DGm3c_55,chr1,3360680,3360680
DGm3c_101,chr1,3488468,3488468
DGm3c_132,chr1,3646772,3646772


In [6]:
motif_hits = anndata.read_h5ad(motif_hits_path)
motif_hits = motif_hits[dmr_bed_df.index, :].copy()
motif_ids = motif_hits.var_names
motif_hits.X = motif_hits.X.todense()
motif_hits

AnnData object with n_obs × n_vars = 46288 × 719 
    obs: 'chrom', 'start', 'end'

In [7]:
# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99, 
                                             0, motif_hits.X),
                         index=motif_hits.var_names)
# only keep value larger than the cutoff for each motif
motif_hits.X = np.multiply(motif_hits.X, (motif_hits.X > motif_cutoff.values[None, :]))

## Background Hits

In [8]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [9]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [10]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids].copy()

In [11]:
judge_matrix = csr_matrix(background_motif_hits.X > motif_cutoff.values[None, :])
background_motif_hits.X = background_motif_hits.X.multiply(judge_matrix)

In [12]:
background_motif_hits

AnnData object with n_obs × n_vars = 337798 × 719 
    obs: 'chrom', 'start', 'end'

## Motif hits contingency table

In [13]:
# calculate motif occurence, not considering hits here
pos = (motif_hits.X > 0).sum(axis=0)
pos_total = motif_hits.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [14]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [15]:
results = {}
with ProcessPoolExecutor(20) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

In [16]:
motif_enrich_df['Group'] = group

In [17]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

21

In [18]:
motif_enrich_df.to_msgpack(output_dir / f'{group}.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [19]:
pybedtools.cleanup()

In [20]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,Group
MA0461.2,2.133484,2.825294e-73,2.901981e-71,70.537305,862.0,45426.0,2978.0,334820.0,0.018623,0.008816,IT-L23+MOs
MA0498.2,1.602599,1.891821e-185,6.801096e-183,182.167421,5617.0,40671.0,26801.0,310997.0,0.121349,0.079340,IT-L23+MOs
MA0507.1,1.678698,5.506868e-34,1.277238e-32,31.893728,760.0,45528.0,3326.0,334472.0,0.016419,0.009846,IT-L23+MOs
MA0607.1,1.758407,1.923351e-134,4.609632e-132,131.336334,2779.0,43509.0,11840.0,325958.0,0.060037,0.035051,IT-L23+MOs
MA0619.1,1.575222,2.670786e-196,1.920295e-193,192.716632,6479.0,39809.0,31633.0,306165.0,0.139971,0.093645,IT-L23+MOs
MA0623.2,2.120482,5.979941e-59,3.908707e-57,56.407967,697.0,45591.0,2418.0,335380.0,0.015058,0.007158,IT-L23+MOs
MA0633.1,1.752825,3.365615e-41,1.273620e-39,38.894960,806.0,45482.0,3381.0,334417.0,0.017413,0.010009,IT-L23+MOs
MA0668.1,2.147961,5.243244e-45,2.513262e-43,42.599762,510.0,45778.0,1743.0,336055.0,0.011018,0.005160,IT-L23+MOs
MA0669.1,2.021532,2.845941e-30,5.683976e-29,28.245348,387.0,45901.0,1403.0,336395.0,0.008361,0.004153,IT-L23+MOs
MA0678.1,2.372703,3.374037e-37,8.664046e-36,35.062279,338.0,45950.0,1044.0,336754.0,0.007302,0.003091,IT-L23+MOs
